# 1. Add Necessary Library

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder
from keras.utils import to_categorical
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Convolution2D, Conv2D, MaxPooling2D, GlobalAveragePooling2D
from keras.optimizers import Adam
from keras.utils import np_utils
from sklearn import metrics
from keras.callbacks import ModelCheckpoint 
from datetime import datetime
from keras.models import model_from_json
import json
import glob
import cv2
import os

Using TensorFlow backend.


# 2. Load Data And Model

In [2]:
#Load ResNet50 Architecture
with open('RESNET\\Resnet50_Architecture.json') as file:#Here, if an error occurs, please follow the absolute path
    # Load model
    architecture = json.load(file)
model = model_from_json(architecture)
model.summary()

Model: "ResNet50"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_7 (InputLayer)            (None, 64, 64, 1)    0                                            
__________________________________________________________________________________________________
zero_padding2d_7 (ZeroPadding2D (None, 70, 70, 1)    0           input_7[0][0]                    
__________________________________________________________________________________________________
conv1 (Conv2D)                  (None, 32, 32, 64)   3200        zero_padding2d_7[0][0]           
__________________________________________________________________________________________________
bn_conv1 (BatchNormalization)   (None, 32, 32, 64)   256         conv1[0][0]                      
___________________________________________________________________________________________

In [ ]:
#Load data
imagePaths = [];data=[];labels=[]

for imagePath in glob.glob('DATA/**/*.jpg', recursive=True):
    imagePaths.append(imagePath)

for imagePath in imagePaths:
    print(imagePath)
    img=cv2.imread(imagePath,0)
    img=np.reshape(img, (64,64,1))
    data.append(img)
    
    label=imagePath.split(os.path.sep)[-2]
    labels.append(label)
    
data = np.array(data, dtype="float")
labels = np.array(labels)

In [ ]:
# Encode the classification labels
lb = LabelEncoder()
yy = to_categorical(lb.fit_transform(labels))

In [ ]:
#Split the dataset
x_train, x_test, y_train, y_test = train_test_split(data, yy, test_size=0.2, random_state = 42)

# 3. Training

In [7]:
from keras.callbacks import ModelCheckpoint

In [8]:
#Create MODEL folder to save Model
os.mkdir("MODEL")
filepath='MODEL/ResNet50.hdf5'
checkpoint = ModelCheckpoint(filepath, monitor='val_loss', verbose=1, save_best_only=True, mode='min', save_weights_only=False)

In [ ]:
# Initialize our initial learning rate and # of epochs to train for
INIT_LR = 0.001
EPOCHS = 50

# Compiling the CNN
opt = Adam(learning_rate=INIT_LR) #decay=INIT_LR/EPOCHS
model.compile(optimizer = opt, loss = 'categorical_crossentropy', metrics = ['accuracy'])

# Fitting the CNN
H = model.fit(x_train, y_train, validation_data=(x_test, y_test), epochs=EPOCHS, batch_size=8, callbacks=[checkpoint])

In [ ]:
from keras.models import load_model
best_model = load_model("MODEL/ResNet50.hdf5")

In [ ]:
# evaluate the network
from sklearn.metrics import classification_report

predictions = best_model.predict(x_test, batch_size=8)

print(classification_report(y_test.argmax(axis=1), predictions.argmax(axis=1), target_names=lb.classes_))

In [ ]:
# plot the training loss and accuracy
N = np.arange(0, EPOCHS)
plt.style.use("ggplot")
plt.figure()
plt.plot(N, H.history["accuracy"], label="train_acc")
plt.plot(N, H.history["val_accuracy"], label="val_acc")
plt.title("Training Accuracy (ResNet)")
plt.xlabel("Epoch")
plt.ylabel("Loss/Accuracy")
plt.legend()
plt.savefig("Result.jpg")